In [1]:
'''
目前有如下几份20220831的数据：
1. 原始特征数据
2. label数据
3. 复推标签数据
4. order time数据

分别读取这几份数据，各自去重，然后合并
'''
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA


In [3]:
# 原始特征数据
raw_feature_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
# label数据
label_data = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
# 复推标签数据
rerec_data = pd.read_csv('../data_20220831/sample_resume_type_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
# order time数据
order_time_data = pd.read_pickle('../generalization_data/cv_jd_time_20220831_82333.pkl')


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_20551/1961729654.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_feature_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_20551/1961729654.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  label_data = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)


In [5]:
print('{}的shape: {}'.format('raw_feature_data', raw_feature_data.shape))
print('{}的shape: {}'.format('label_data', label_data.shape))
print('{}的shape: {}'.format('rerec_data', rerec_data.shape))
print('{}的shape: {}'.format('order_time_data', order_time_data.shape))

raw_feature_data的shape: (82333, 86)
label_data的shape: (82333, 128)
rerec_data的shape: (82333, 3)
order_time_data的shape: (82333, 5)


In [16]:
label_data.columns

Index(['cv_id', 'jd_id', 'cv_code', 'jd_code', 'last_opt', 'da_label',
       'rank_label', 'title', 'deadline', 'category_id',
       ...
       'industry_cut', 'industry_jd2vec', 'industry_vec2', 'majorName_cut',
       'majorName_jd2vec', 'majorName_vec2', 'skills_cut', 'skills_jd2vec',
       'skills_vec2', 'ds'],
      dtype='object', length=128)

In [13]:
rerec_data.columns

Index(['cv_id', 'jd_id', 'resume_type'], dtype='object')

In [14]:
order_time_data.columns

Index(['cv_id', 'jd_id', 'pair_created_time', 'day', 'month'], dtype='object')

In [17]:
# 合并上述表的关键信息，raw feature暂时不用考虑
cv_jd_raw_label_rank_label = label_data[['cv_id', 'jd_id', 'da_label', 'rank_label']]
cv_jd_resume_type = rerec_data[['cv_id', 'jd_id', 'resume_type']]
cv_jd_pair_time = order_time_data[['cv_id', 'jd_id', 'month']].rename(columns={'month': 'order_month'})

In [21]:
res = cv_jd_raw_label_rank_label.\
            join(cv_jd_resume_type.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
            join(cv_jd_pair_time.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
res

,cv_id,jd_id,da_label,rank_label,resume_type,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,1,1,NaN,2021-09
1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,-1,1,4.0,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,1,1,4.0,2022-03
3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,2,2,NaN,2021-10
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,3,3,NaN,2021-10
...,...,...,...,...,...,...
82328,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,-1,1,5.0,2022-08
82329,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,-1,2,4.0,2022-07
82330,8a69d7c282199453018247bb00cb68a3,8a69c468826ee9b40182831bedfc5e79,3,3,4.0,2022-08
82331,8a69d7c282199453018247bb00cb68a3,8a69f6f8823a93b9018247c30aca4e3d,-1,2,4.0,2022-07
